In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import shutil
# import cv2
from scipy import ndimage
# from skimage.transform import resize
from joblib import Parallel, delayed
import tensorflow as tf
from sklearn.model_selection import KFold

2022-11-07 13:46:01.616365: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# get the names of the models in the model folder
model_names = os.listdir("/home/statsgeneral/gayara/Tasselnet/Filtered_90/models")

In [3]:
# how many models?
len(model_names)

50

In [4]:
model_names[0]

'model_bootstrap26.h5'

In [5]:
loaded_models = []
for model in model_names:
    model_ld = tf.keras.models.load_model("/home/statsgeneral/gayara/Tasselnet/Filtered_90/models/" + model)
    loaded_models.append(model_ld)

2022-11-07 13:46:04.162306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 13:46:04.726919: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30965 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0


In [6]:
# names of the test files
numpy_folder = "/home/statsgeneral/gayara/Tasselnet/Filtered_90/np_test_files"
test_files = os.listdir(numpy_folder)

In [7]:
# how many test files
len(test_files)

65

In [8]:
small_list = loaded_models[:2]

In [9]:
len(small_list)

2

In [10]:
# Function to get the predictions given model and image

numpy_folder = "/home/statsgeneral/gayara/Tasselnet/Filtered_90/np_test_files"
v_stack_folder = "/home/statsgeneral/gayara/Tasselnet/Filtered_90/test_files_stack"

def prediction_on_test_data(model, selected_file, stride = 8, kernel_size = 32):
#     load the cnn model
    
# load the image data file
    load_image = np.load(numpy_folder + "/"+ selected_file)
    
    # get the image height
    img_height = load_image.shape[0]
    # get the image weight
    img_width = load_image.shape[1]

    all_test_sub_windows = np.load(v_stack_folder + "/"+ selected_file)

    # now, to get the predictions, pass the sub windows
    test_image_prediction = model.predict(all_test_sub_windows, batch_size = 100)
    
    # density map
    Density_map = np.zeros((img_height, img_width))

    # counts map
    counts_map = np.zeros((img_height, img_width))
    
    # now, for every window, we will keep adding the values together and also add the counts
    counter = 0
#     need a counter to move into each predicted value in the pred values list
    for ii in range(0, img_height, stride):
        for jj in range(0, img_width, stride):
#         operations for density map
#             get the window of interest
            new_window = Density_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     fill each with the value c_k
            counts_window = np.full((new_window.shape[0], new_window.shape[1]), test_image_prediction[counter])
#     get the shapes of this new window
            cw_height = counts_window.shape[0]
            cw_width = counts_window.shape[1]
#         Do c_k/r_2
            counts_window_new = counts_window/(cw_height*cw_width)
#     This is the value in the window now
            value_window = counts_window_new
#     place the values in the corrsponding area of the density map
            Density_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window + value_window

#         Let's now focus on capturing the counts of the windows
            new_window_c = counts_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     get the counts area
            count = np.ones((new_window_c.shape[0], new_window_c.shape[1]))
#     keep adding the counts to reflect the addition of densities
            counts_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window_c + count
#     increase the counter
            counter = counter + 1
            
#         get the normalized count
    normalized_counts = np.divide(Density_map, counts_map)
    
#     entire count on the test set
    pred_on_test = np.sum(normalized_counts)
    
#     return the predicted value
    return(pred_on_test, normalized_counts, selected_file)


In [11]:
model_names[:2]

['model_bootstrap26.h5', 'model_bootstrap31.h5']

In [12]:
# get the preds and normalized counts

all_preds = []
for mdl in loaded_models:
    preds_sub = []
    for file in test_files:
        results = prediction_on_test_data(mdl, file)
        preds_sub.append(results)
    all_preds.append(preds_sub)

2022-11-07 13:46:10.549083: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


34/63 [===============>..............] - ETA: 0s  

2022-11-07 13:46:11.077840: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-07 13:46:11.078217: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-07 13:46:11.078228: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-07 13:46:11.078482: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-07 13:46:11.078519: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


68/68 [==============================] - 0s 1ms/step


In [13]:
all_preds[1][64][1].shape

(1305, 327)

In [14]:
# get the true counts for the 65 test set images
# import the csv file to see how it looks like
coords_data = pd.read_csv(r"C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\new_annotated_coordinates_all.csv", header = None)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\15739\\Desktop\\UNL - all\\Fall 2022\\Manuscript\\Filtered_90\\new_annotated_coordinates_all.csv'

In [11]:
coords_data.head()

,0,1,2,3,4
0,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,42,45,61,40
1,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,40,84,65,41
2,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,34,119,76,36
3,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,71,430,45,50
4,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,59,614,35,46


In [12]:
# add column names
coords_data.columns = ['Image_name', 'top_x', 'top_y', 'width', 'height']

In [13]:
coords_data.head()

,Image_name,top_x,top_y,width,height
0,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,42,45,61,40
1,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,40,84,65,41
2,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,34,119,76,36
3,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,71,430,45,50
4,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,59,614,35,46


In [14]:
len(coords_data['Image_name'].unique())

323

In [15]:
# how many coords in each image? 
# These are the true counts
true_counts_df = pd.DataFrame(coords_data.groupby('Image_name').size()).reset_index()

In [16]:
# see the dataset
true_counts_df.head()

,Image_name,0
0,190812_djifc6310s_Aug12_NE_row_16_range_4_rep_...,32
1,190812_djifc6310s_Aug12_NE_row_18_range_4_rep_...,25
2,190812_djifc6310s_Aug12_NE_row_33_range_2_rep_...,51
3,190812_djifc6310s_Aug12_NW_row_17_range_4_rep_...,36
4,190812_djifc6310s_Aug12_NW_row_22_range_2_rep_...,38


In [17]:
test_image_names = [i.split('.')[0] for i in test_files]

In [18]:
test_image_names_final = [i + '.png' for i in test_image_names]

In [19]:
len(test_image_names)

65

In [20]:
# select the images in the test folder only
test_true_counts = true_counts_df.loc[true_counts_df['Image_name'].isin(test_image_names_final)].reset_index(drop = True)

In [21]:
# how many images here?
test_true_counts.shape

(65, 2)

In [22]:
# rename the column 0 as true counts
test_true_counts.columns = ['Image_name', "True_count"]

In [23]:
test_true_counts.head()

,Image_name,True_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50


In [24]:
# save this dataframe for future use
test_true_counts.to_csv('true_counts_test_only_sample20.csv', index = False)

In [25]:
preds_list = []
files_list = []
for i in all_preds:
    prediction = i[0]
    files = i[2]
    preds_list.append(prediction)
    files_list.append(files)


In [26]:
files_list_png = [x.split('.')[0] + '.png' for x in files_list]

In [27]:
# d = {'Image name': Image_name, 'Predicted value': pred_data}
preds_df  = list(zip(files_list_png, preds_list))

In [28]:
preds_df = pd.DataFrame(preds_df)

In [29]:
preds_df.columns = ['Image_name', 'Predicted_value']

In [30]:
preds_df.head()

,Image_name,Predicted_value
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,35.562046
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,25.229440
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,26.034970
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,37.463887
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,41.643262


In [31]:
# merge the true and predicted counts
final_data = pd.merge(test_true_counts, preds_df, on = 'Image_name')

In [32]:
final_data.shape

(65, 3)

In [33]:
mae = np.mean(abs(final_data['True_count'] - final_data['Predicted_value'] ))

In [34]:
mae

9.104540829614649

In [35]:
final_data.to_csv('model1_preds_bootstrap.csv', index = False)

In [36]:
# need to also save the normalized counts
norm_counts = []
for i in all_preds:
    count = i[1]
    norm_counts.append(count)

In [37]:
bb_app = []
for i in range(len(norm_counts)):
    bb = np.save(r'C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\codes_and_models\norm_counts_1_bootstrap' + '\\' + files_list[i], norm_counts[i])
    bb_app.append(bb)

Preds for model 2

In [38]:
model2 = loaded_models[1]

In [39]:
# get the preds and normalized counts

all_preds_model2 = []
for file in tqdm(test_files):
    results = prediction_on_test_data(model2, file)
    all_preds_model2.append(results)

  0%|          | 0/65 [00:00<?, ?it/s]

In [40]:
preds_list_md2 = []
files_list_md2 = []
for i in all_preds_model2:
    prediction = i[0]
    files = i[2]
    preds_list_md2.append(prediction)
    files_list_md2.append(files)


In [41]:
files_list_png = [x.split('.')[0] + '.png' for x in files_list_md2]

In [42]:
# d = {'Image name': Image_name, 'Predicted value': pred_data}
preds_df  = list(zip(files_list_png, preds_list_md2))

In [43]:
preds_df = pd.DataFrame(preds_df)

In [44]:
preds_df.columns = ['Image_name', 'Predicted_value']

In [45]:
preds_df.head()

,Image_name,Predicted_value
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,33.988139
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,24.473418
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,24.987135
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,35.988741
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,39.335704


In [46]:
# import the test true counts preciously stored
test_true = pd.read_csv("true_counts_test_only.csv")

In [47]:
test_true.head()

,Image_name,True_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50


In [48]:
# merge the true and predicted counts
final_data = pd.merge(test_true, preds_df, on = 'Image_name')

In [49]:
final_data.shape

(65, 3)

In [50]:
mae = np.mean(abs(final_data['True_count'] - final_data['Predicted_value'] ))

In [51]:
mae

8.474501933373354

In [52]:
final_data.to_csv('model2_preds_bootstrap.csv', index = False)

In [53]:
# need to also save the normalized counts
norm_counts = []
for i in all_preds_model2:
    count = i[1]
    norm_counts.append(count)

In [54]:
bb_app = []
for i in range(len(norm_counts)):
    bb = np.save(r'C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\codes_and_models\norm_counts_2_bootstrap' + '\\' + files_list_md2[i], norm_counts[i])
    bb_app.append(bb)

#### Preds for model 3

In [55]:
model3 = loaded_models[2]

In [56]:
# get the preds and normalized counts

all_preds_model3 = []
for file in tqdm(test_files):
    results = prediction_on_test_data(model3, file)
    all_preds_model3.append(results)

  0%|          | 0/65 [00:00<?, ?it/s]

In [57]:
preds_list_md3 = []
files_list_md3 = []
for i in all_preds_model3:
    prediction = i[0]
    files = i[2]
    preds_list_md3.append(prediction)
    files_list_md3.append(files)

In [58]:
files_list_png = [x.split('.')[0] + '.png' for x in files_list_md3]

In [59]:
# d = {'Image name': Image_name, 'Predicted value': pred_data}
preds_df  = list(zip(files_list_png, preds_list_md3))

In [60]:
preds_df = pd.DataFrame(preds_df)

In [61]:
preds_df.columns = ['Image_name', 'Predicted_value']

In [62]:
preds_df.head()

,Image_name,Predicted_value
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,34.984967
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,24.583193
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,25.144224
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,36.243447
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,40.233363


In [63]:
# import the test true counts preciously stored
test_true = pd.read_csv("true_counts_test_only.csv")

In [64]:
test_true.head()

,Image_name,True_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50


In [65]:
# merge the true and predicted counts
final_data = pd.merge(test_true, preds_df, on = 'Image_name')

In [66]:
final_data.shape

(65, 3)

In [67]:
mae = np.mean(abs(final_data['True_count'] - final_data['Predicted_value'] ))

In [68]:
mae

8.439817443540921

In [69]:
final_data.to_csv('model3_preds_bootstrap.csv', index = False)

In [70]:
# need to also save the normalized counts
norm_counts3 = []
for i in all_preds_model3:
    count = i[1]
    norm_counts3.append(count)

In [71]:
bb_app = []
for i in range(len(norm_counts3)):
    bb = np.save(r'C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\codes_and_models\norm_counts_3_bootstrap' + '\\' + files_list_md3[i], norm_counts3[i])
    bb_app.append(bb)

#### Preds for model 4

In [72]:
model4 = loaded_models[3]

In [73]:
# get the preds and normalized counts

all_preds_model4 = []
for file in tqdm(test_files):
    results = prediction_on_test_data(model4, file)
    all_preds_model4.append(results)

  0%|          | 0/65 [00:00<?, ?it/s]

In [74]:
preds_list_md4 = []
files_list_md4 = []
for i in all_preds_model4:
    prediction = i[0]
    files = i[2]
    preds_list_md4.append(prediction)
    files_list_md4.append(files)


In [75]:
files_list_png = [x.split('.')[0] + '.png' for x in files_list_md4]

In [76]:
# d = {'Image name': Image_name, 'Predicted value': pred_data}
preds_df  = list(zip(files_list_png, preds_list_md4))

In [77]:
preds_df = pd.DataFrame(preds_df)

In [78]:
preds_df.columns = ['Image_name', 'Predicted_value']

In [79]:
preds_df.head()

,Image_name,Predicted_value
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,37.049322
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,25.483444
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,26.565284
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,38.167156
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,45.459402


In [80]:
# import the test true counts preciously stored
test_true = pd.read_csv("true_counts_test_only.csv")

In [81]:
test_true.head()

,Image_name,True_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50


In [82]:
# merge the true and predicted counts
final_data = pd.merge(test_true, preds_df, on = 'Image_name')

In [83]:
final_data.shape

(65, 3)

In [84]:
mae = np.mean(abs(final_data['True_count'] - final_data['Predicted_value'] ))

In [85]:
mae

9.72901790449876

In [86]:
final_data.to_csv('model4_preds_bootstrap.csv', index = False)

In [87]:
# need to also save the normalized counts
norm_counts4 = []
for i in all_preds_model4: 
    count = i[1]
    norm_counts4.append(count)

In [88]:
bb_app = []
for i in range(len(norm_counts4)):
    bb = np.save(r'C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\codes_and_models\norm_counts_4_bootstrap' + '\\' + files_list_md4[i], norm_counts4[i])
    bb_app.append(bb)

#### Model 5 Preds

In [89]:
model5 = loaded_models[4]

In [90]:
# get the preds and normalized counts

all_preds_model5 = []
for file in tqdm(test_files):
    results = prediction_on_test_data(model5, file)
    all_preds_model5.append(results)

  0%|          | 0/65 [00:00<?, ?it/s]

In [91]:
preds_list_md5 = []
files_list_md5 = []
for i in all_preds_model5:
    prediction = i[0]
    files = i[2]
    preds_list_md5.append(prediction)
    files_list_md5.append(files)


In [92]:
files_list_png = [x.split('.')[0] + '.png' for x in files_list_md5]

In [93]:
# d = {'Image name': Image_name, 'Predicted value': pred_data}
preds_df  = list(zip(files_list_png, preds_list_md5))

In [94]:
preds_df = pd.DataFrame(preds_df)

In [95]:
preds_df.columns = ['Image_name', 'Predicted_value']

In [96]:
preds_df.head()

,Image_name,Predicted_value
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,34.464187
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,23.664466
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,24.242424
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,36.343543
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,40.121906


In [97]:
# import the test true counts preciously stored
test_true = pd.read_csv("true_counts_test_only.csv")

In [98]:
test_true.head()

,Image_name,True_count
0,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
1,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
2,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
3,190812_djifc6310s_Aug12_SW_row_11_range_2_rep_...,33
4,190814_djifc6310s_Aug14_NE_row_33_range_2_rep_...,50


In [99]:
# merge the true and predicted counts
final_data = pd.merge(test_true, preds_df, on = 'Image_name')

In [100]:
final_data.shape

(65, 3)

In [101]:
mae = np.mean(abs(final_data['True_count'] - final_data['Predicted_value'] ))

In [102]:
mae

8.320746798844878

In [103]:
final_data.to_csv('model5_preds_bootstrap.csv', index = False)

In [104]:
# need to also save the normalized counts
norm_counts5 = []
for i in all_preds_model5:
    count = i[1]
    norm_counts5.append(count)

In [105]:
bb_app = []
for i in range(len(norm_counts5)):
    bb = np.save(r'C:\Users\15739\Desktop\UNL - all\Fall 2022\Manuscript\Filtered_90\codes_and_models\norm_counts_5_bootstrap' + '\\' + files_list_md5[i], norm_counts5[i])
    bb_app.append(bb)